In [2]:
import numpy as np 
import pandas as pd 

In [69]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#Model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [5]:
data = [pd.read_csv(f'dataset-of-{decade}0s.csv') for decade in ['6', '7', '8', '9', '0', '1']]

In [6]:
for i, decade in enumerate([1960, 1970, 1980, 1990, 2000, 2010]):
    data[i]['decade'] = pd.Series(decade, index=data[i].index)

data = pd.concat(data, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [7]:
from pandas_profiling import ProfileReport
prof = ProfileReport(data)
prof.to_file(output_file='output.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# EDA

In [8]:
data.head()

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target,decade
0,Attaining - Take 1 / Alternate Version,John Coltrane,spotify:track:3EwLV5hZqLKx5e0Lp1QcB7,0.342,0.462,4,-12.931,0,0.0389,0.51400,0.018100,0.0977,0.397,98.466,816867,4,24.84938,33,0,1970
1,So Fly,NB Ridaz Featuring Gemini,spotify:track:2BjIi07kN0yKSur0Fwrnss,0.861,0.519,2,-6.404,1,0.1120,0.13600,0.000000,0.1230,0.519,81.966,254533,4,50.03590,11,1,2000
2,Because I Got It Like That,Jungle Brothers,spotify:track:5unLExF3iiG3YkU11u6wFO,0.900,0.916,1,-7.481,0,0.1150,0.22000,0.000141,0.0323,0.538,102.916,277894,4,24.71271,16,0,1980
3,Babylon a Fall - Remastered,Yabby You,spotify:track:6xfe0G2HwRDQaChxkzvNKw,0.714,0.301,2,-14.800,1,0.1230,0.15600,0.661000,0.2290,0.651,74.856,357671,4,104.64231,11,0,1970
4,Fins,Jimmy Buffett,spotify:track:4h0gZ422QxBRdTV14u0P8y,0.661,0.645,4,-13.520,1,0.0487,0.00895,0.000000,0.0362,0.930,136.693,204640,4,31.99617,10,1,1970


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41106 entries, 0 to 41105
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track             41106 non-null  object 
 1   artist            41106 non-null  object 
 2   uri               41106 non-null  object 
 3   danceability      41106 non-null  float64
 4   energy            41106 non-null  float64
 5   key               41106 non-null  int64  
 6   loudness          41106 non-null  float64
 7   mode              41106 non-null  int64  
 8   speechiness       41106 non-null  float64
 9   acousticness      41106 non-null  float64
 10  instrumentalness  41106 non-null  float64
 11  liveness          41106 non-null  float64
 12  valence           41106 non-null  float64
 13  tempo             41106 non-null  float64
 14  duration_ms       41106 non-null  int64  
 15  time_signature    41106 non-null  int64  
 16  chorus_hit        41106 non-null  float6

In [10]:
#Check Any Null Value
pd.isnull(data).sum()

track               0
artist              0
uri                 0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
chorus_hit          0
sections            0
target              0
decade              0
dtype: int64

**Univariate Analysis of every column**

In [11]:
print(data['target'].value_counts())

0    20553
1    20553
Name: target, dtype: int64


In [73]:
# Plot the distribution of the target variable
sns.countplot(x='target', data=data)
plt.show()

In [ ]:
# calculate the proportion of target values that are 1
prop_1 = data[data['target'] == 1].shape[0] / data.shape[0]
print("Proportion of target values that are 1:", prop_1)

Proportion of target values that are 1: 0.5


In [15]:
# calculate the mean of other columns for observations where target is 1
print(data.loc[data['target'] == 1].mean())

danceability             0.601237
energy                   0.624296
key                      5.248528
loudness                -8.702237
mode                     0.730064
speechiness              0.069444
acousticness             0.280814
instrumentalness         0.030687
liveness                 0.192637
valence                  0.609578
tempo                  120.288280
duration_ms         226095.539970
time_signature           3.938063
chorus_hit              39.224034
sections                10.183380
target                   1.000000
decade                1982.775264
dtype: float64


C:\Users\m\AppData\Local\Temp\ipykernel_16056\603395626.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(data.loc[data['target'] == 1].mean())


In [74]:
# calculate the mean of other columns for observations where target is 0
print(data.loc[data['target'] == 0].mean())

danceability             0.478153
energy                   0.534794
key                      5.178660
loudness               -11.740813
mode                     0.656644
speechiness              0.076477
acousticness             0.447581
instrumentalness         0.278145
liveness                 0.210432
valence                  0.475302
tempo                  118.388219
duration_ms         243659.560940
time_signature           3.849316
chorus_hit              40.988048
sections                10.767966
target                   0.000000
decade                1982.775264
year                  1982.775264
hit                      0.000000
dtype: float64


In [75]:
# create a box plot to compare the distribution of other columns between target 0 and target 1
columns_to_compare = ['danceability', 'energy', 'loudness', 'valence']
for column in columns_to_compare:
    sns.boxplot(x='target', y=column, data=data)
    plt.show()

In [18]:
# get summary statistics of the danceability column
print(data['danceability'].describe())

count    41106.000000
mean         0.539695
std          0.177821
min          0.000000
25%          0.420000
50%          0.552000
75%          0.669000
max          0.988000
Name: danceability, dtype: float64


In [19]:
# Plot the distribution of the danceability variable
sns.distplot(data['danceability'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\164479068.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['danceability'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\164479068.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [20]:
# check for outliers in the danceability column
sns.boxplot(x=data['danceability'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\3219671053.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [21]:
# get summary statistics of the energy column
print(data['energy'].describe())


count    41106.000000
mean         0.579545
std          0.252628
min          0.000251
25%          0.396000
50%          0.601000
75%          0.787000
max          1.000000
Name: energy, dtype: float64


In [22]:
# Plot the distribution of the energy variable
sns.distplot(data['energy'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\399186095.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['energy'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\399186095.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [23]:
# check for outliers in the energy column
sns.boxplot(x=data['energy'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\2279068549.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [24]:
# Plot the distribution of the loudness variable
sns.distplot(data['loudness'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\2736522257.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['loudness'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\2736522257.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [25]:
# Plot the distribution of the loudness variable
sns.distplot(data['speechiness'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\12964489.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['speechiness'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\12964489.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [26]:
# Plot the distribution of the loudness variable
sns.distplot(data['acousticness'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\1654057172.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['acousticness'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\1654057172.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [27]:
# Plot the distribution of the loudness variable
sns.distplot(data['instrumentalness'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\3892087779.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['instrumentalness'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\3892087779.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [28]:
# Plot the distribution of the loudness variable
sns.distplot(data['liveness'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\3322644786.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['liveness'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\3322644786.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [29]:
# Plot the distribution of the loudness variable
sns.distplot(data['valence'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\1806393977.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['valence'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\1806393977.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [30]:
# Plot the distribution of the loudness variable
sns.distplot(data['tempo'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\3387701840.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['tempo'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\3387701840.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [31]:
# Plot the distribution of the loudness variable
sns.distplot(data['duration_ms'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\281239526.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['duration_ms'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\281239526.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [32]:
# Plot the distribution of the loudness variable
sns.distplot(data['time_signature'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\3869395968.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['time_signature'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\3869395968.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [33]:
# Plot the distribution of the loudness variable
sns.distplot(data['chorus_hit'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\346298354.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['chorus_hit'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\346298354.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [34]:
# Plot the distribution of the loudness variable
sns.distplot(data['decade'])
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\586522974.py:2: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(data['decade'])
C:\Users\m\AppData\Local\Temp\ipykernel_16056\586522974.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


**Bivariate Analysis**

In [35]:
# create a scatter plot of danceability vs energy
sns.scatterplot(x='danceability', y='energy', data=data)
plt.show()


C:\Users\m\AppData\Local\Temp\ipykernel_16056\2503621619.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [36]:
# create a bar plot of the average danceability by decade
data.groupby('decade')['danceability'].mean().plot(kind='bar')
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\1890931697.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [37]:
# create a violin plot of danceability by target
sns.violinplot(x='target', y='danceability', data=data)
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\1751103547.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [38]:
# create a bar plot of the average energy by decade
data.groupby('decade')['energy'].mean().plot(kind='bar')
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\3378577333.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [39]:
# create a violin plot of energy by target
sns.violinplot(x='target', y='energy', data=data)
plt.show()


C:\Users\m\AppData\Local\Temp\ipykernel_16056\748412127.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [40]:
# Create a box plot for the relationship between loudness and target
sns.boxplot(x='target', y='loudness', data=data)
plt.show()


C:\Users\m\AppData\Local\Temp\ipykernel_16056\3148864300.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [41]:
# Create a bar plot for the relationship between key and target
sns.barplot(x='key', y='target', data=data)
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\4113287589.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [42]:
# Create a violin plot for the relationship between mode and speechiness
sns.violinplot(x='mode', y='speechiness', hue='target', data=data)
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\2276316577.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


**What is the average danceability of songs that were hits?**

In [43]:
hit_danceability = data[data['target'] == 1]['danceability'].mean()
print("The average danceability of songs that were hits: ", hit_danceability)


The average danceability of songs that were hits:  0.6012373765387048


**What is the average energy of songs that were not hits?**

In [44]:
not_hit_energy = data[data['target'] == 0]['energy'].mean()
print("The average energy of songs that were not hits: ", not_hit_energy)


The average energy of songs that were not hits:  0.5347942697416436


**What is the average speechiness of songs from the 2000s?**

In [45]:
speechiness_2000s = data[data['decade'] == 2000]['speechiness'].mean()
print("The average speechiness of songs from the 2000s: ", speechiness_2000s)

The average speechiness of songs from the 2000s:  0.09235953678474114


**What is the correlation between instrumentalness and liveness for songs that were not hits?**

In [46]:
corr = data[data['target'] == 0][['instrumentalness', 'liveness']].corr()
print("The correlation between instrumentalness and liveness for songs that were not hits: ", corr)

The correlation between instrumentalness and liveness for songs that were not hits:                    instrumentalness  liveness
instrumentalness          1.000000 -0.106062
liveness                 -0.106062  1.000000


**What is the average tempo of songs that were hits?**

In [47]:
hit_tempo = data[data['target'] == 1]['tempo'].mean()
print("The average tempo of songs that were hits: ", hit_tempo)

The average tempo of songs that were hits:  120.28827971585658


**What is the median loudness of songs that were not hits?**

In [48]:
not_hit_loudness = data[data['target'] == 0]['loudness'].median()
print("The median loudness of songs that were not hits: ", not_hit_loudness)

The median loudness of songs that were not hits:  -10.664


**year with the highest hits**

In [49]:
data['year'] = data['decade'] + (data['decade'] % 10)

data['hit'] = data['target'].apply(lambda x: 1 if x == 1 else 0)

yearly_hits = data.groupby('year')['hit'].sum()
year_with_highest_hits = yearly_hits.idxmax()
print("The year with the highest number of hits: ", year_with_highest_hits)

sns.barplot(x=yearly_hits.index, y=yearly_hits.values)
plt.xlabel("Year")
plt.ylabel("Number of Hits")
plt.show()

The year with the highest number of hits:  1960


C:\Users\m\AppData\Local\Temp\ipykernel_16056\3526289443.py:12: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


**top 3 difference between popular song and non-popular song by audio features?**

In [50]:
df4 = data[data['target'] == 1]

# Calculate and print the mean value of audio feature for popular song
print(df4.mean())

# Filter the audio features data by non-popular song 

df5 = data[data['target'] == 0]

# Calculate and print the mean value of audio feature for non-popular song
print(df5.mean())

# Show the value difference between popular songs and non popular songs by printing
print(df4.mean()-df5.mean())

# Plot the bar graph
df6 = df4.mean()-df5.mean()

df6.plot(
  kind = 'bar',
  title = 'The Top Difference Between Popular Song and Non-Popular Song by Audio Features',
  figsize = [12,8],
  color = 'orange'
  )

C:\Users\m\AppData\Local\Temp\ipykernel_16056\2303848260.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df4.mean())


danceability             0.601237
energy                   0.624296
key                      5.248528
loudness                -8.702237
mode                     0.730064
speechiness              0.069444
acousticness             0.280814
instrumentalness         0.030687
liveness                 0.192637
valence                  0.609578
tempo                  120.288280
duration_ms         226095.539970
time_signature           3.938063
chorus_hit              39.224034
sections                10.183380
target                   1.000000
decade                1982.775264
year                  1982.775264
hit                      1.000000
dtype: float64


C:\Users\m\AppData\Local\Temp\ipykernel_16056\2303848260.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df5.mean())


danceability             0.478153
energy                   0.534794
key                      5.178660
loudness               -11.740813
mode                     0.656644
speechiness              0.076477
acousticness             0.447581
instrumentalness         0.278145
liveness                 0.210432
valence                  0.475302
tempo                  118.388219
duration_ms         243659.560940
time_signature           3.849316
chorus_hit              40.988048
sections                10.767966
target                   0.000000
decade                1982.775264
year                  1982.775264
hit                      0.000000
dtype: float64


C:\Users\m\AppData\Local\Temp\ipykernel_16056\2303848260.py:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df4.mean()-df5.mean())


danceability            0.123085
energy                  0.089501
key                     0.069868
loudness                3.038576
mode                    0.073420
speechiness            -0.007033
acousticness           -0.166767
instrumentalness       -0.247458
liveness               -0.017796
valence                 0.134276
tempo                   1.900061
duration_ms        -17564.020970
time_signature          0.088746
chorus_hit             -1.764014
sections               -0.584586
target                  1.000000
decade                  0.000000
year                    0.000000
hit                     1.000000
dtype: float64


C:\Users\m\AppData\Local\Temp\ipykernel_16056\2303848260.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df6 = df4.mean()-df5.mean()


<AxesSubplot: title={'center': 'The Top Difference Between Popular Song and Non-Popular Song by Audio Features'}, xlabel='Year', ylabel='Number of Hits'>

**top 5 artists with the highest number of hit songs in the dataset**

In [51]:
data['hit'] = data['target'].apply(lambda x: 1 if x == 1 else 0)
artist_hits = data.groupby('artist')['hit'].sum().sort_values(ascending=False)

top_5_artists = artist_hits.head(5)
print("The top 5 artists with the highest number of hits: \n", top_5_artists)

# Plot the number of hits per artist
top_5_artists.plot(kind='bar')
plt.xlabel("Artist")
plt.ylabel("Number of Hits")
plt.show()

The top 5 artists with the highest number of hits: 
 artist
The Beatles           59
Elton John            56
The Rolling Stones    54
Stevie Wonder         53
Madonna               53
Name: hit, dtype: int64


C:\Users\m\AppData\Local\Temp\ipykernel_16056\2749072434.py:11: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


**What are the top 4 keys with the highest liveness?**

In [52]:
top_4_keys_liveness = data.sort_values(by='liveness',
ascending=False).groupby('key').mean().sort_values(by='liveness', ascending=False).head(4)
print("The top 4 keys with the highest liveness: \n", top_4_keys_liveness)

The top 4 keys with the highest liveness: 
      danceability    energy   loudness      mode  speechiness  acousticness  \
key                                                                           
4        0.522066  0.586986 -10.147801  0.554017     0.066214      0.358393   
7        0.539817  0.571788 -10.400520  0.804111     0.069582      0.372331   
11       0.560496  0.629182  -9.574045  0.460391     0.083116      0.283540   
6        0.556014  0.626970  -9.337614  0.557059     0.087804      0.290990   

     instrumentalness  liveness   valence       tempo    duration_ms  \
key                                                                    
4            0.145684  0.216126  0.532034  119.771573  233835.265423   
7            0.151022  0.208603  0.551720  118.734045  231149.281137   
11           0.147583  0.206594  0.547247  120.111876  240285.809947   
6            0.133232  0.202778  0.542199  119.035699  239758.919260   

     time_signature  chorus_hit   sections    ta

In [53]:
sns.pairplot(data)
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\411007246.py:2: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [54]:
# Plot a heatmap of the correlation matrix for the dataset
sns.heatmap(data.corr(), annot=True)
plt.show()

C:\Users\m\AppData\Local\Temp\ipykernel_16056\1192267710.py:3: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Preprocessing

In [55]:
data.head()

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target,decade,year,hit
0,Attaining - Take 1 / Alternate Version,John Coltrane,spotify:track:3EwLV5hZqLKx5e0Lp1QcB7,0.342,0.462,4,-12.931,0,0.0389,0.51400,...,0.397,98.466,816867,4,24.84938,33,0,1970,1970,0
1,So Fly,NB Ridaz Featuring Gemini,spotify:track:2BjIi07kN0yKSur0Fwrnss,0.861,0.519,2,-6.404,1,0.1120,0.13600,...,0.519,81.966,254533,4,50.03590,11,1,2000,2000,1
2,Because I Got It Like That,Jungle Brothers,spotify:track:5unLExF3iiG3YkU11u6wFO,0.900,0.916,1,-7.481,0,0.1150,0.22000,...,0.538,102.916,277894,4,24.71271,16,0,1980,1980,0
3,Babylon a Fall - Remastered,Yabby You,spotify:track:6xfe0G2HwRDQaChxkzvNKw,0.714,0.301,2,-14.800,1,0.1230,0.15600,...,0.651,74.856,357671,4,104.64231,11,0,1970,1970,0
4,Fins,Jimmy Buffett,spotify:track:4h0gZ422QxBRdTV14u0P8y,0.661,0.645,4,-13.520,1,0.0487,0.00895,...,0.930,136.693,204640,4,31.99617,10,1,1970,1970,1


In [56]:
# Drop Categorical Data

data.drop(['track', 'artist','uri'],axis=1,inplace=True)

In [57]:
#Train - Test Split
y = data['target']
X = data.drop('target', axis = 1)

In [58]:
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,decade,year,hit
0,0.342,0.462,4,-12.931,0,0.0389,0.51400,0.018100,0.0977,0.397,98.466,816867,4,24.84938,33,1970,1970,0
1,0.861,0.519,2,-6.404,1,0.1120,0.13600,0.000000,0.1230,0.519,81.966,254533,4,50.03590,11,2000,2000,1
2,0.900,0.916,1,-7.481,0,0.1150,0.22000,0.000141,0.0323,0.538,102.916,277894,4,24.71271,16,1980,1980,0
3,0.714,0.301,2,-14.800,1,0.1230,0.15600,0.661000,0.2290,0.651,74.856,357671,4,104.64231,11,1970,1970,0
4,0.661,0.645,4,-13.520,1,0.0487,0.00895,0.000000,0.0362,0.930,136.693,204640,4,31.99617,10,1970,1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41101,0.435,0.834,10,-10.762,1,0.0415,0.78900,0.422000,0.9320,0.708,114.130,263067,4,17.81915,10,1960,1960,0
41102,0.499,0.880,6,-6.424,0,0.0654,0.01080,0.000028,0.2260,0.390,147.899,264800,4,55.42304,11,2000,2000,1
41103,0.326,0.816,8,-14.825,1,0.0527,0.88100,0.326000,0.1850,0.826,147.213,259267,4,25.73316,13,1960,1960,1
41104,0.659,0.257,9,-16.552,1,0.0382,0.89000,0.000140,0.2520,0.222,102.977,228160,4,20.91621,13,1970,1970,1


In [59]:
y

0        0
1        1
2        0
3        0
4        1
        ..
41101    0
41102    1
41103    1
41104    1
41105    1
Name: target, Length: 41106, dtype: int64

In [60]:


#Split train-test data into 7:3 (for RFC)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, shuffle = True, random_state = 1)


#Scale 

X_train = pd.DataFrame(preprocessing.robust_scale(X_train), index=X_train.index)
X_test = pd.DataFrame(preprocessing.robust_scale(X_test), index=X_test.index)

In [61]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
17085,-0.618474,-0.795918,-0.500000,-1.057926,0.0,-0.302521,1.016923,14.046411,-0.341332,-0.378657,0.487920,1.302007,0.0,0.105797,1.50,-0.333333,-0.333333,-1.0
5559,-1.365462,0.895408,0.166667,0.783520,-1.0,1.512605,-0.293012,0.468992,0.779844,-0.371773,0.271877,-0.332228,0.0,2.473598,-1.25,0.000000,0.000000,-1.0
32888,-0.200803,0.568878,-0.833333,0.539484,0.0,-0.131653,0.289878,-0.001976,1.769646,0.580608,0.951156,0.504895,0.0,-0.576751,0.75,0.000000,0.000000,-1.0
14170,-0.871486,0.895408,-0.166667,0.456700,-1.0,0.473389,-0.401124,4.223831,0.911818,-0.112450,-0.528102,1.023342,0.0,1.455086,1.25,0.666667,0.666667,-1.0
7958,0.421687,-0.867347,-0.833333,0.030755,0.0,-0.425770,0.466938,-0.001976,0.083983,-0.658635,-0.885180,-0.672603,0.0,-0.314002,-0.75,-0.333333,-0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7813,0.714859,0.992347,0.666667,0.715073,-1.0,0.159664,-0.385146,-0.001258,1.865627,0.647160,0.223301,1.434375,0.0,-0.877128,1.50,0.000000,0.000000,0.0
32511,-0.931727,0.326531,-0.500000,-0.256292,-1.0,3.380952,0.054842,-0.001891,0.275945,-0.270797,1.480700,2.583747,0.0,1.185997,1.25,-0.333333,-0.333333,-1.0
5192,-0.630522,-0.806122,1.000000,-1.285467,-1.0,-0.252101,0.040740,-0.001956,0.485903,-0.087206,-0.580378,-0.667782,0.0,-0.085333,-0.50,0.333333,0.333333,-1.0
12172,0.092369,0.818878,-0.833333,0.540255,-1.0,0.733894,-0.398668,7.659315,3.221356,-0.690763,0.109131,1.054127,0.0,-0.117160,-1.00,0.666667,0.666667,-1.0


In [62]:
y_train

17085    0
5559     0
32888    0
14170    0
7958     1
        ..
7813     1
32511    0
5192     0
12172    0
33003    1
Name: target, Length: 28774, dtype: int64

In [63]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.


C:\Users\m\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.


In [64]:
for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))

                   Logistic Regression: 49.97%
                   K-Nearest Neighbors: 70.55%
                         Decision Tree: 49.97%
Support Vector Machine (Linear Kernel): 49.97%
   Support Vector Machine (RBF Kernel): 51.47%
                        Neural Network: 49.97%
                         Random Forest: 49.97%
                     Gradient Boosting: 49.97%


In [65]:
clf=RandomForestClassifier()

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [66]:
import joblib

In [67]:
joblib.dump(clf, 'clf.pkl')

['clf.pkl']